In [1]:
# Loading library
import re
import os
import nltk
from nltk.collocations import *
from itertools import chain
import string
import itertools
from bs4 import BeautifulSoup
from html import unescape
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams 
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import *
import urllib
import pandas as pd
import glob
from collections import Counter
import os

In [2]:
# set to enviroment to reproducible neural network
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['PYTHONHASHSEED']=str(0)

## Extracting the dataset

In [3]:
import xml.etree.ElementTree as et

def parse_XML(xml_file): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    document=''
    for doc in xroot[0]:
        document+=doc.text+'.'
        
    return document

In [4]:
from zipfile import ZipFile
zf = ZipFile('./Assessment2_data/data.zip', 'r')
zf.extractall('./Assessment2_data/')

In [5]:
dict_author= {}
path = './Assessment2_data/data'
for filename in glob.glob(os.path.join(path, '*.xml')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: 
        dict_author[re.search('\w*.xml',f.name)[0][:-4]]=parse_XML(f)
        

In [6]:
id_order=[]
doc_order=[]
for k,v in dict_author.items():
    id_order.append(k)
    doc_order.append(v)

In [7]:
df_train=pd.read_csv('./Assessment2_data/train_labels.csv')
df=pd.DataFrame({'id' : id_order,'doc' : doc_order},columns=['id', 'doc'])
df=pd.merge(df_train, df, how='outer', on='id')

## Preprocessing
The following basic pre-processing steps include
Text cleaning: Remove emoticons,symbols, metion,hashtag

Case normalization: Text can contain upper- or lowercase letters. It is a good idea to just allow either uppercase or lowercase.

Tokenization is the process of splitting a stream of text into individual words.

Stopwords are words that are extremely common and carry little lexical content. 

Removing number letter and token which has less than 3 character.

Removing the most/least frequent word: Besides the stopwords, we usually remove words appearing in more than 95% of the documents and less than 3% of the documents as well. 


In [8]:
def clean_doc(doc):
        emoji_pattern = re.compile("["
             u"\U0001F600-\U0001F64F"  # emoticons
             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
             u"\U0001F680-\U0001F6FF"  # transport & map symbols
             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
             u"\U00002702-\U000027B0"
             u"\U000024C2-\U0001F251"
             "]+", flags=re.UNICODE)
        #HappyEmoticons
        emoticons_happy = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
        ])
        # Sad Emoticons
        emoticons_sad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
        ])
        #combine sad and happy emoticons
        emoticons = emoticons_happy.union(emoticons_sad)
        #
        #
        segdoc=sent_tokenize(doc)
        clean_doc=''
        for sentence in segdoc:
            # remove mention
            sentence=re.sub('@\w*', '', sentence)
            # remove hashtag
            sentence = re.sub('#', '', sentence)
            # remove retweet
            sentence = re.sub('RT[\s]+', '', sentence) 
            # remove hyperlink
            sentence = re.sub('https?:\/\/\S+', '', sentence)
            # remove special string of html content
            sentence=BeautifulSoup(unescape(sentence)).text

            # remove icon
            for icon in emoticons:
                sentence=sentence.replace(icon,'')
            # remove emoji
            sentence=emoji_pattern.sub(r'', sentence)

            sentence = re.sub(r"what's", "what is ", sentence)
            sentence = re.sub(r"\'s", " ", sentence)
            sentence = re.sub(r"\'ve", " have ", sentence)
            sentence = re.sub(r"can't", "cannot ", sentence)
            sentence = re.sub(r"n't", " not ", sentence)
            sentence = re.sub(r"i'm", "i am ", sentence)
            sentence = re.sub(r"\'re", " are ", sentence)
            sentence = re.sub(r"\'d", " would ", sentence)
            sentence = re.sub(r"\'ll", " will ", sentence)
            sentence = re.sub(r"\'scuse", " excuse ", sentence)
            sentence = re.sub('\W', ' ', sentence)
            sentence = re.sub('\s+', ' ', sentence)
            sentence=sentence.strip(' ')
            clean_doc+=sentence+ ' '
        return clean_doc

In [9]:
def doc_tokenize(doc):
    #Emoji patterns
    tokenizer = RegexpTokenizer(r"[A-Za-z0-9]\w*(?:['?]\w+)?")
    tokens=[]
    tokens+=tokenizer.tokenize(clean_doc(doc))
    # lower case token
    tokens=[w.lower() for w in tokens]
    return tokens

In [10]:
def filter_tokens(doc):
    stop_words = set(stopwords.words('english'))
    tokens=doc_tokenize(doc)
    filtered_tokens=[]
    # remove punctation
    for w in tokens:
        if w not in string.punctuation and w not in stop_words and len(w)>=3 and not(w.isnumeric()) :
            filtered_tokens.append(w)
    return filtered_tokens

In [11]:
for i in range(len(df)):
    df.iloc[i,2]=filter_tokens(df.iloc[i,2])

/usr/local/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [12]:
for i in range(len(df)):
    df.iloc[i,2]=' '.join(df.iloc[i,2])

# Feature extraction 

There are 2 features often considered in document classification, which include

• Bi-gram feature

• Unigram feature

We will vectorize thoes feature by TFIDF and Word2Vec to reduce dimension. 

In [13]:
test=df[df['gender'].isnull()]
train=df[~df['gender'].isnull()]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(token_pattern=r"\w+['\w]*",ngram_range=(1,2),min_df=0.01,max_df=0.95) # specify parameters here
tfidf_model.fit(train['doc'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=0.01, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern="\\w+['\\w]*", tokenizer=None,
                use_idf=True, vocabulary=None)

In [15]:
raw_train_tfidf=tfidf_model.transform(train['doc'])
raw_test_tfidf=tfidf_model.transform(test['doc'])

In [16]:
#Word embedding
#Glove Vectors
vocab=tfidf_model.vocabulary_
embeddings_index = {}
f = open('./crawl-300d-2M.vec')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Total 1999996 word vectors.


In [17]:
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))
for word, i in vocab.items():
    if word in embeddings_index.keys():
        embedding_matrix[i] = embeddings_index.get(word)

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 1302


In [18]:
trainsum=np.sum(raw_train_tfidf.toarray(),axis=1)
w2v_train_tfidf=raw_train_tfidf.dot(embedding_matrix)
for i in range(len(train)):
    w2v_train_tfidf[i]=w2v_train_tfidf[i]/trainsum[i]

In [19]:
testsum=np.sum(raw_test_tfidf.toarray(),axis=1)
w2v_test_tfidf=raw_test_tfidf.dot(embedding_matrix)
for i in range(len(test)):
    w2v_test_tfidf[i]=w2v_test_tfidf[i]/testsum[i]

# Random forest

Fine tuning to select best hyper-parameters of random forest

In [20]:
from sklearn import pipeline,ensemble,preprocessing,feature_extraction,metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from numpy.random import seed
from tensorflow.python.framework import ops
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
import tensorflow as tf
import random as rn 

Using TensorFlow backend.


In [21]:
kfold=model_selection.KFold(n_splits=5, shuffle=True, random_state=1)

In [22]:
# set seed to reproducible random forest
forest_model=ensemble.RandomForestClassifier(verbose=1,n_jobs=-1,random_state=1)
parameters={}
parameters['n_estimators']=[100,200,300,400,500,600,700,800,900,1000]
parameters['max_depth']=[10,20,30,40,50,60,70,80,90,100]
parameters['min_samples_split']=[2,5,10]
parameters['max_features']=['sqrt','log2']

Cv_rf=RandomizedSearchCV(forest_model,parameters,cv=kfold,n_jobs=-1,n_iter = 20,random_state=1)
Cv_rf.fit(w2v_train_tfidf,train['gender'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    5.2s finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                   error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_lea...
                                                    random_state=1, verbos

In [23]:
Cv_rf.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'max_features': 'sqrt',
 'max_depth': 30}

# Neural Network

Fine tuning to select best hyper-parameters for neural network

In [24]:
def create_model(n1=100,a='tanh',d=300):
    model = Sequential()
    model.add(Dense(n1,kernel_initializer='normal', input_dim=d,activation=a))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid')) 
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

In [25]:
neural_model=KerasClassifier(build_fn=create_model,verbose=1,d=300)
parameters={}
parameters['n1']=[50,100,120,150,200]
parameters['a']=['relu','tanh']
parameters['epochs']=[15,30,50]
parameters['batch_size']=[10,20,40,80]

In [26]:
# set seed in tensor flow to reproducible neural network
ops.reset_default_graph()
np.random.seed(1)   
rn.seed(2)   
tf.random.set_seed(3)

Cv_neural=RandomizedSearchCV(neural_model,parameters,cv=kfold,n_jobs=1,n_iter=20,random_state=1)
Cv_neural.fit(w2v_train_tfidf,train['gender'])

Epoch 1/30
2480/2480 [==============================] - 0s 115us/step - loss: 0.6782 - accuracy: 0.6145
Epoch 2/30
2480/2480 [==============================] - 0s 34us/step - loss: 0.6180 - accuracy: 0.7254
Epoch 3/30
2480/2480 [==============================] - 0s 28us/step - loss: 0.5474 - accuracy: 0.7625
Epoch 4/30
2480/2480 [==============================] - 0s 25us/step - loss: 0.5021 - accuracy: 0.7746
Epoch 5/30
2480/2480 [==============================] - 0s 25us/step - loss: 0.4760 - accuracy: 0.7851
Epoch 6/30
2480/2480 [==============================] - 0s 27us/step - loss: 0.4592 - accuracy: 0.7940
Epoch 7/30
2480/2480 [==============================] - 0s 24us/step - loss: 0.4471 - accuracy: 0.7931
Epoch 8/30
2480/2480 [==============================] - 0s 24us/step - loss: 0.4400 - accuracy: 0.8056
Epoch 9/30
2480/2480 [==============================] - 0s 24us/step - loss: 0.4330 - accuracy: 0.8032
Epoch 10/30
2480/2480 [==============================] - 0s 27us/step - 

Epoch 19/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.4128 - accuracy: 0.8153
Epoch 20/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.4085 - accuracy: 0.8113
Epoch 21/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.4073 - accuracy: 0.8129
Epoch 22/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.4066 - accuracy: 0.8157
Epoch 23/30
2480/2480 [==============================] - 0s 19us/step - loss: 0.4050 - accuracy: 0.8258
Epoch 24/30
2480/2480 [==============================] - 0s 21us/step - loss: 0.4040 - accuracy: 0.8173
Epoch 25/30
2480/2480 [==============================] - 0s 21us/step - loss: 0.3996 - accuracy: 0.8230
Epoch 26/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.3994 - accuracy: 0.8238
Epoch 27/30
2480/2480 [==============================] - 0s 20us/step - loss: 0.4013 - accuracy: 0.8165
Epoch 28/30
2480/2480 [==============================] - 0s 20us

2480/2480 [==============================] - 0s 16us/step - loss: 0.3813 - accuracy: 0.8315
Epoch 25/30
2480/2480 [==============================] - 0s 15us/step - loss: 0.3828 - accuracy: 0.8310
Epoch 26/30
2480/2480 [==============================] - 0s 14us/step - loss: 0.3759 - accuracy: 0.8351
Epoch 27/30
2480/2480 [==============================] - 0s 15us/step - loss: 0.3749 - accuracy: 0.8335
Epoch 28/30
2480/2480 [==============================] - 0s 15us/step - loss: 0.3687 - accuracy: 0.8399
Epoch 29/30
2480/2480 [==============================] - 0s 15us/step - loss: 0.3660 - accuracy: 0.8383
Epoch 30/30
620/620 [==============================] - 0s 36us/step
Epoch 1/30
2480/2480 [==============================] - 0s 56us/step - loss: 0.6792 - accuracy: 0.6153
Epoch 2/30
2480/2480 [==============================] - 0s 19us/step - loss: 0.6308 - accuracy: 0.6960
Epoch 3/30
2480/2480 [==============================] - 0s 17us/step - loss: 0.5729 - accuracy: 0.7403
Epoch 4/30


2480/2480 [==============================] - 0s 17us/step - loss: 0.3587 - accuracy: 0.8452
Epoch 40/50
2480/2480 [==============================] - 0s 16us/step - loss: 0.3629 - accuracy: 0.8415
Epoch 41/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3560 - accuracy: 0.8476
Epoch 42/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3533 - accuracy: 0.8500
Epoch 43/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3547 - accuracy: 0.8492
Epoch 44/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3576 - accuracy: 0.8508
Epoch 45/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3504 - accuracy: 0.8516
Epoch 46/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3519 - accuracy: 0.8516
Epoch 47/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3467 - accuracy: 0.8565
Epoch 48/50
2480/2480 [==============================] - 0s 15us/step - loss

Epoch 17/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.4021 - accuracy: 0.8165
Epoch 18/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.4045 - accuracy: 0.8153
Epoch 19/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3967 - accuracy: 0.8198
Epoch 20/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3943 - accuracy: 0.8238
Epoch 21/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3955 - accuracy: 0.8206
Epoch 22/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3982 - accuracy: 0.8218
Epoch 23/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3851 - accuracy: 0.8286
Epoch 24/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3850 - accuracy: 0.8282
Epoch 25/50
2480/2480 [==============================] - 0s 15us/step - loss: 0.3833 - accuracy: 0.8278
Epoch 26/50
2480/2480 [==============================] - 0s 16us

...

Epoch 21/50
2480/2480 [==============================] - 0s 25us/step - loss: 0.3979 - accuracy: 0.8250
Epoch 22/50
2480/2480 [==============================] - 0s 25us/step - loss: 0.3916 - accuracy: 0.8302
Epoch 23/50
2480/2480 [==============================] - 0s 25us/step - loss: 0.3970 - accuracy: 0.8278
Epoch 24/50
2480/2480 [==============================] - 0s 24us/step - loss: 0.3892 - accuracy: 0.8294
Epoch 25/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3930 - accuracy: 0.8242
Epoch 26/50
2480/2480 [==============================] - 0s 33us/step - loss: 0.3910 - accuracy: 0.8282
Epoch 27/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3877 - accuracy: 0.8319
Epoch 28/50
2480/2480 [==============================] - 0s 24us/step - loss: 0.3864 - accuracy: 0.8367
Epoch 29/50
2480/2480 [==============================] - 0s 24us/step - loss: 0.3894 - accuracy: 0.8274
Epoch 30/50
2480/2480 [==============================] - 0s 25us

Epoch 27/50
2480/2480 [==============================] - 0s 27us/step - loss: 0.3854 - accuracy: 0.8310
Epoch 28/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3790 - accuracy: 0.8363
Epoch 29/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3825 - accuracy: 0.8371
Epoch 30/50
2480/2480 [==============================] - 0s 25us/step - loss: 0.3853 - accuracy: 0.8323
Epoch 31/50
2480/2480 [==============================] - 0s 25us/step - loss: 0.3810 - accuracy: 0.8359
Epoch 32/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3774 - accuracy: 0.8375
Epoch 33/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3783 - accuracy: 0.8371
Epoch 34/50
2480/2480 [==============================] - 0s 27us/step - loss: 0.3762 - accuracy: 0.8379
Epoch 35/50
2480/2480 [==============================] - 0s 26us/step - loss: 0.3739 - accuracy: 0.8399
Epoch 36/50
2480/2480 [==============================] - 0s 27us

2480/2480 [==============================] - 0s 87us/step - loss: 0.4010 - accuracy: 0.8294
Epoch 12/30
2480/2480 [==============================] - 0s 88us/step - loss: 0.3952 - accuracy: 0.8327
Epoch 13/30
2480/2480 [==============================] - 0s 85us/step - loss: 0.3996 - accuracy: 0.8290
Epoch 14/30
2480/2480 [==============================] - 0s 85us/step - loss: 0.3918 - accuracy: 0.8306
Epoch 15/30
2480/2480 [==============================] - 0s 84us/step - loss: 0.3990 - accuracy: 0.8246
Epoch 16/30
2480/2480 [==============================] - 0s 85us/step - loss: 0.3888 - accuracy: 0.8274
Epoch 17/30
2480/2480 [==============================] - 0s 85us/step - loss: 0.3951 - accuracy: 0.8278
Epoch 18/30
2480/2480 [==============================] - 0s 88us/step - loss: 0.3991 - accuracy: 0.8242
Epoch 19/30
2480/2480 [==============================] - 0s 89us/step - loss: 0.3867 - accuracy: 0.8290
Epoch 20/30
2480/2480 [==============================] - 0s 87us/step - loss

Epoch 13/15
2480/2480 [==============================] - 0s 82us/step - loss: 0.4026 - accuracy: 0.8177
Epoch 14/15
2480/2480 [==============================] - 0s 81us/step - loss: 0.3985 - accuracy: 0.8169
Epoch 15/15
620/620 [==============================] - 0s 68us/step
Epoch 1/15
2480/2480 [==============================] - 0s 135us/step - loss: 0.6105 - accuracy: 0.6794
Epoch 2/15
2480/2480 [==============================] - 0s 99us/step - loss: 0.4877 - accuracy: 0.7786
Epoch 3/15
2480/2480 [==============================] - 0s 109us/step - loss: 0.4587 - accuracy: 0.7940
Epoch 4/15
2480/2480 [==============================] - 0s 125us/step - loss: 0.4422 - accuracy: 0.8016
Epoch 5/15
2480/2480 [==============================] - 0s 131us/step - loss: 0.4304 - accuracy: 0.8101
Epoch 6/15
2480/2480 [==============================] - 0s 131us/step - loss: 0.4254 - accuracy: 0.8089
Epoch 7/15
2480/2480 [==============================] - 0s 111us/step - loss: 0.4167 - accuracy: 0.81

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                   error_score='raise-deprecating',
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x183e32e90>,
                   iid='warn', n_iter=20, n_jobs=1,
                   param_distributions={'a': ['relu', 'tanh'],
                                        'batch_size': [10, 20, 40, 80],
                                        'epochs': [15, 30, 50],
                                        'n1': [50, 100, 120, 150, 200]},
                   pre_dispatch='2*n_jobs', random_state=1, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [27]:
Cv_neural.best_params_

{'n1': 200, 'epochs': 30, 'batch_size': 80, 'a': 'relu'}

# Logistic regression

Fine tuning select best hyper-parameter for logistic regression

In [29]:
# set random_state to reproducible logistic regression
lg_model=LogisticRegression(random_state=1)
parameters={}
parameters['C']=[.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
parameters['penalty']=['l2','l1']
parameters['max_iter']=[50,100,150]

In [30]:
# Model develop on second feature
Cv_lg=GridSearchCV(lg_model,parameters,cv=kfold,n_jobs=-1)
Cv_lg.fit(w2v_train_tfidf,train['gender'])

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=1, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5],
                         'max_iter': [50, 100, 150], 'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [31]:
Cv_lg.best_params_

{'C': 5, 'max_iter': 50, 'penalty': 'l2'}

# Voting classifier
Fine tunning to select best hyper-parameter for Voting classifier

In [32]:
ops.reset_default_graph()
np.random.seed(1)   
rn.seed(2)   
tf.random.set_seed(3)
voting = VotingClassifier(estimators=[('rf',Cv_rf.best_estimator_), ('lg',Cv_lg.best_estimator_),
                                     ('neural',Cv_neural.best_estimator_)],
        voting="soft",weights=[1,2,3],flatten_transform=True)

parameters={}
parameters['weights']=[[1,2,3],[1,3,2],[3,2,1],[1,1,1]]

Cv_voting=GridSearchCV(voting,parameters,cv=kfold,n_jobs=1)
Cv_voting.fit(w2v_train_tfidf,train['gender'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    3.3s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Epoch 1/30
2480/2480 [==============================] - 0s 53us/step - loss: 0.6795 - accuracy: 0.6415
Epoch 2/30
2480/2480 [==============================] - 0s 16us/step - loss: 0.6312 - accuracy: 0.7226
Epoch 3/30
2480/2480 [==============================] - 0s 15us/step - loss: 0.5699 - accuracy: 0.7532
Epoch 4/30
2480/2480 [==============================] - 0s 16us/step - loss: 0.5174 - accuracy: 0.7718
Epoch 5/30
2480/2480 [==============================] - 0s 14us/step - loss: 0.4834 - accuracy: 0.7802
Epoch 6/30
2480/2480 [==============================] - 0s 13us/step - loss: 0.4666 - accuracy: 0.7891
Epoch 7/30
2480/2480 [==============================] - 0s 13us/step - loss: 0.4498 - accuracy: 0.7956
Epoch 8/30
2480/2480 [==============================] - 0s 13us/step - loss: 0.4415 - accuracy: 0.8077
Epoch 9/30
2480/2480 [==============================] - 0s 13us/step - loss: 0.4346 - accuracy: 0.8032
Epoch 10/30
2480/2480 [==============================] - 0s 13us/step - l

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 800 out of 800 | elapsed:    0.1s finished


620/620 [==============================] - 0s 25us/step


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    3.5s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 800 out of 800 | elapsed:    0.1s finished


620/620 [==============================] - 0s 16us/step


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    4.6s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Epoch 1/30
3100/3100 [==============================] - 0s 45us/step - loss: 0.6738 - accuracy: 0.6065
Epoch 2/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.6093 - accuracy: 0.7165
Epoch 3/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.5416 - accuracy: 0.7613
Epoch 4/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.5004 - accuracy: 0.7719
Epoch 5/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.4761 - accuracy: 0.7874
Epoch 6/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4609 - accuracy: 0.7932
Epoch 7/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4527 - accuracy: 0.7977
Epoch 8/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4423 - accuracy: 0.7977
Epoch 9/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4373 - accuracy: 0.8013
Epoch 10/30
3100/3100 [==============================] - 0s 14us/step - l

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score='raise-deprecating',
             estimator=VotingClassifier(estimators=[('rf',
                                                     RandomForestClassifier(bootstrap=True,
                                                                            class_weight=None,
                                                                            criterion='gini',
                                                                            max_depth=30,
                                                                            max_features='sqrt',
                                                                            max_leaf_nodes=None,
                                                                            min_impurity_decrease=0.0,
                                                                            min_impurity_split=None,
                                                             

In [33]:
Cv_voting.best_params_

{'weights': [1, 3, 2]}

In [47]:
ops.reset_default_graph()
np.random.seed(1)   
rn.seed(2)   
tf.random.set_seed(3)

Cv_voting.best_estimator_.fit(w2v_train_tfidf,train['gender'])
test['gender']=Cv_voting.best_estimator_.predict(w2v_test_tfidf)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    4.0s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Epoch 1/30
3100/3100 [==============================] - 0s 47us/step - loss: 0.6723 - accuracy: 0.6323
Epoch 2/30
3100/3100 [==============================] - 0s 18us/step - loss: 0.6053 - accuracy: 0.7229
Epoch 3/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.5387 - accuracy: 0.7610
Epoch 4/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4976 - accuracy: 0.7826
Epoch 5/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4784 - accuracy: 0.7810
Epoch 6/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4681 - accuracy: 0.7868
Epoch 7/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4572 - accuracy: 0.7919
Epoch 8/30
3100/3100 [==============================] - 0s 14us/step - loss: 0.4463 - accuracy: 0.8019
Epoch 9/30
3100/3100 [==============================] - 0s 15us/step - loss: 0.4364 - accuracy: 0.8026
Epoch 10/30
3100/3100 [==============================] - 0s 14us/step - l

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 800 out of 800 | elapsed:    0.1s finished
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
df_test=pd.read_csv('./Assessment2_data/test_labels.csv')
df_test=pd.merge(df_test, test, how='inner', on='id')
cm=confusion_matrix(df_test['gender_x'],df_test['gender_y'],labels=['female','male'])
tn, fp, fn, tp=cm.ravel()
print('Test TPR: ', tp/(tp+fn))
print('Test TNR: ', tn/(tn+fp))
print('Test Accuracy: ' ,accuracy_score(df_test['gender_x'],df_test['gender_y']))
test1=test.drop(['doc'],axis=1)

Test TPR:  0.8024193548387096
Test TNR:  0.8373015873015873
Test Accuracy:  0.82


In [51]:
test1.to_csv('pred_labels.csv',index=False)